# Práctica 1: Análisis estadístico de la Regla de Lipinski

> **Note:** Este libro esta disponible de dos maneras: 
> 1. Descargando el repositorio y siguiendo las instrucciones que estan en el archivo [README.md](https://github.com/ramirezlab/PILE/blob/main/README.md)
> 2. Haciendo clic aquí en [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ramirezlab/PILE/blob/main/2.%20De%20datos%20a%20gráficas%3A%20Propiedades%20drug-likeness%20y%20similitud%20química%20con%20python/2.3_Practica-1.es.ipynb?hl=es)

## Conceptos a trabajar

### **Farmacocinética**

La farmacocinética es el estudio de lo que le sucede a un compuesto en un organismo durante un período de tiempo<sup> **1** </sup> . Se divide en cuatro pasos: **A**bsorción, **D**istribución, **M**etabolismo y **E**xcreción (ADME)**<sup> 1, </sup>** **<sup> 2 </sup>**. En ocasiones también se incluye **T**oxicología (ADMET) y **L**iberación (LADME).

 
<img src="img/ADME-es.jpg" alt="ADME" width="800"/>

*Figura 1*. Pasos que componen la farmacocinética. Adaptada de: [Somvanshi, Kharat, Jadhav, Thorat y Townley, 2021](https://doi.org/10.1016/B978-0-323-85050-6.00007-4)

   * **Absorción:** Se refiere a la cantidad y el tiempo que tarda un compuesto o sustancia en ingresar a la circulación sistémica desde el sitio de administración. Depende de múltiples factores como la capacidad del compuesto para penetrar la pared intestinal, la solubilidad del compuesto, el tiempo de vaciado gástrico, la estabilidad química del compuesto en el estómago, entre otros<sup> **1, 2** </sup>.
   * **Distribución:** Se refiere a cómo una sustancia se distribuye por todo el cuerpo. Depende de la difusión y la convección, que pueden verse influidas por la polaridad, el tamaño o la capacidad de unión del fármaco, el estado de líquidos del paciente o la constitución corporal del individuo. Es muy importante lograr la concentración eficaz del fármaco en el sitio del receptor porque, para ser eficaz, un medicamento debe llegar a su destino compartimental designado<sup> **1,2** </sup>.
   * **Metabolismo:** Se refiere al procesamiento del fármaco por parte del cuerpo en compuestos posteriores. También puede ser convertir un fármaco en sustancias más solubles en agua para que sea más fácil de excretar o, en el caso de los profármacos, se requiere el metabolismo para convertir el fármaco en metabolitos activos<sup> **1,2** </sup>.
   * **Excreción:** Se refiere al proceso por el cual el fármaco es eliminado del organismo. Generalmente, los riñones son conductos de excreción por filtración pasiva en los glomérulos o secreción en los túbulos<sup> **1, 2** </sup>.

### **Reglas de Lipinski:**

Las reglas de Lipinski es una forma de descartar compuestos con probables problemas de absorción. Esta regla establece que la mala absorción o penetración de un fármaco es más probable cuando la estructura química cumple con dos o más de los siguientes criterios<sup> **3** </sup>:
1. El peso molecular (MW) es superior a 500.
2. El valor de log P calculado es superior a 5.
3. Hay más de 5 donantes de enlaces de hidrógeno (–NH–, –OH).
4. El número de aceptores de enlaces de hidrógeno (–N ¼ , –O–) es mayor que 10.

Es importante saber que la regla de cinco no categoriza definitivamente todos los compuestos bien y mal absorbidos, aunque es simple, rápida y proporciona un grado razonable de clasificación.

## Planteamiento del problema

Para una investigación de un nuevo fármaco queremos saber si realmente es absorbido por el cuerpo, si es capaz de cruzar ciertas barreras para llegar a su objetivo, cómo se metaboliza y cómo se excreta del cuerpo. De esta manera, los médicos tendrán mayor flexibilidad en la prescripción y administración de medicamentos, brindando así mayor beneficio con menor riesgo y haciendo los ajustes necesarios, dada la variada fisiología y estilos de vida de los pacientes.

Para conocer la absorción de los compuestos utilizaremos herramientas bioinformáticas para poder calcular la regla de cinco de Lipinski y luego calcularemos unos estadísticos para graficarlos y analizarlos.

## Importar las bibliotecas necesarias

In [2]:
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd
from rdkit.Chem import Draw
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from math import pi
import os
from pathlib import Path

## Cargar conjunto de datos de P49841
El conjunto de datos contiene los compuestos bioactivos contra la glucógeno sintasa quinasa-3 beta que construimos en el tutorial 2.1_Dataframes.
Lo primero que vamos a hacer es importar la base de datos, tenemos que crear un `directorio raíz` (`ROOT_DIR`) para poder navegar hasta el archivo.

In [ ]:
# Importa la librería pandas para la manipulación de datos
import pandas as pd

# URL del archivo CSV que contiene los datos
csv_url = 'https://raw.githubusercontent.com/ramirezlab/PILE/main/2.%20De%20datos%20a%20gráficas%3A%20Propiedades%20drug-likeness%20y%20similitud%20química%20con%20python/data/compounds_P49841_full.csv'

# Carga el archivo CSV desde la URL en un DataFrame de pandas
df_output = pd.read_csv(csv_url)

# Muestra las primeras 5 filas del DataFrame para verificar la correcta importación de los datos
df_output.head()

## Las reglas de Lipinski (Ro5)

La siguiente función nos permitirá calcular las propiedades químicas de la regla de cinco de Lipinski teniendo como entrada los `SMILES`. Luego se definirán las condiciones de la regla de cinco y finalmente tendremos información de si se viola la regla de cinco.

In [ ]:
def Ro5(df):
    
    smi = df['smiles']
    m = Chem.MolFromSmiles(smi)
    
    # Calcular la regla de cinco propiedades químicas
    MW = Descriptors.ExactMolWt(m)
    HBA = Descriptors.NumHAcceptors(m)
    HBD = Descriptors.NumHDonors(m)
    LogP = Descriptors.MolLogP(m)
    
    # Regla de cinco condiciones
    conditions = [MW <= 500, HBA <= 10, HBD <= 5, LogP <= 5]
    
    # Crear una fila de pandas para resultados de condiciones con valores e información sobre si se viola la regla de cinco
    return pd.Series([MW, HBA, HBD, LogP, 'yes']) if conditions.count(True) >= 3 else pd.Series([MW, HBA, HBD, LogP, 'no'])

Ahora vamos a aplicar las Ro5 a nuestro conjunto de datos

In [ ]:
# Aplica la función Ro5 a cada fila del DataFrame df_output y almacena el resultado en df_rule5
df_rule5 = df_output.apply(Ro5, axis=1)

# Asigna nombres a las columnas del DataFrame df_rule5
df_rule5.columns = ['MW', 'HBA', 'HBD', 'LogP', 'rule_of_five_conform']

# Muestra las primeras 5 filas del DataFrame actualizado
df_rule5.head()

In [ ]:
# Une el DataFrame df_output con df_rule5 agregando las columnas generadas por la regla de los cinco (Ro5)
df_molecule = df_output.join(df_rule5)

# Muestra las primeras 5 filas del DataFrame actualizado
df_molecule.head()

In [ ]:
# Filtrar el DataFrame para mantener solo los compuestos que cumplen con la regla de cinco (Ro5)
fil_df = df_molecule[df_molecule['rule_of_five_conform'] == 'yes']

# Imprimir el número total de compuestos en el DataFrame original
print('# de compuestos:', len(df_molecule))

# Imprimir el número de compuestos que cumplen con la regla de cinco
print('# de compuestos en conjunto de datos filtrados:', len(fil_df))

# Imprimir el número de compuestos que NO cumplen con la regla de cinco de Lipinski
print("# de compuestos que no cumplen con la regla de cinco de Lipinski:", (len(df_molecule) - len(fil_df)))

# Contar y mostrar cuántos compuestos cumplen y cuántos no cumplen la regla de cinco
print(df_molecule.rule_of_five_conform.value_counts())

# Generar un gráfico de barras con la distribución de compuestos que cumplen y no cumplen la regla de cinco
df_molecule.rule_of_five_conform.value_counts().plot.bar()

In [ ]:
# Crea un directorio llamado 'data/' si no existe
!mkdir -p data/

# Guarda el DataFrame df_molecule en un archivo CSV dentro del directorio 'data/'
df_molecule.to_csv('data/compounds_P49841_lipinski.csv', index=False)

## Graficar las propiedades de la regla de cinco por molécula como gráficos de barras.

In [ ]:
# Importar el conjunto de datos desde la URL y cargarlo en un DataFrame de pandas
lipinski_comp = pd.read_csv('https://raw.githubusercontent.com/ramirezlab/PILE/main/2.%20De%20datos%20a%20gráficas%3A%20Propiedades%20drug-likeness%20y%20similitud%20química%20con%20python/data/compounds_P49841_lipinski.csv')

# Mostrar las primeras 10 filas del DataFrame para verificar la correcta importación de los datos
lipinski_comp.head(10)

In [ ]:
# Seleccionar las primeras 5 filas del DataFrame lipinski_comp
comp_5_lipinski = lipinski_comp.iloc[:5]

# Mostrar el subconjunto de los primeros 5 compuestos
comp_5_lipinski

#### Ahora haremos el gráfico de barras.

In [ ]:
# Definir un diccionario con las propiedades de la regla de cinco de Lipinski
ro5_properties = {
    "MW": (500, "molecular weight (g/mol)"),  # Peso molecular máximo permitido: 500 g/mol
    "HBA": (10, "# HBA"),  # Máximo 10 aceptores de enlaces de hidrógeno (HBA)
    "HBD": (5, "# HBD"),  # Máximo 5 donantes de enlaces de hidrógeno (HBD)
    "LogP": (5, "logP"),  # Coeficiente de partición logP máximo de 5
}


In [ ]:
# Crear una figura con 4 subgráficas en una sola fila (1 row, 4 columns)
fig, axes = plt.subplots(figsize=(10, 2.5), nrows=1, ncols=4)

# Crear un array con las posiciones en el eje X para los 5 compuestos seleccionados
x = np.arange(1, len(comp_5_lipinski) + 1)

# Definir colores para las barras de cada compuesto
colors = ["DarkMagenta", "LightGreen", "blue", "DarkSalmon", "yellow"]

# Crear subgráficas para cada propiedad de la regla de cinco (Ro5)
for index, (key, (threshold, title)) in enumerate(ro5_properties.items()):
    # Generar gráfico de barras para cada propiedad de la Ro5
    axes[index].bar([0, 1, 2, 3, 4], comp_5_lipinski[key], color=colors)
    
    # Agregar una línea de referencia en la posición del umbral definido por la Ro5
    axes[index].axhline(y=threshold, color="black", linestyle="dashed")
    
    # Asignar título a la subgráfica con el nombre de la propiedad
    axes[index].set_title(title)
    
    # Remover las etiquetas del eje X para mayor claridad
    axes[index].set_xticks([])

# Crear una leyenda con los identificadores de las moléculas y la línea de umbral
legend_elements = [mpatches.Patch(color=color, label=row["molecule_chembl_id"]) 
                   for color, (_, row) in zip(colors, comp_5_lipinski.iterrows())]

# Agregar la línea de referencia a la leyenda
legend_elements.append(Line2D([0], [0], color="black", ls="dashed", label="Threshold"))

# Posicionar la leyenda fuera de la figura para mejor visualización
fig.legend(handles=legend_elements, bbox_to_anchor=(1.2, 0.8))

# Ajustar el diseño de las subgráficas para evitar superposición
plt.tight_layout()

# Mostrar la figura
plt.show()


## Graficar las propiedades de la regla de cinco por molécula como diagramas de dispersión.

In [ ]:
# Crear una figura de tamaño 20x20 para mejorar la visualización de los gráficos
fig = plt.figure(figsize=(20, 20))

# Generar un gráfico de pares (pairplot) usando Seaborn
ax = sns.pairplot(
    data=lipinski_comp,  # Usar el DataFrame con los compuestos filtrados
    vars=['HBD', 'HBA', 'MW', 'LogP'],  # Seleccionar las variables a comparar
    hue='rule_of_five_conform'  # Colorear los puntos según si cumplen la regla de cinco o no
)

# Mostrar el gráfico
plt.show()

# Cerrar la figura para liberar memoria
plt.close()

## Graficar las propiedades de la regla de cinco por molécula como gráfico de radar.

In [ ]:
# Importar el conjunto de datos
lipinski_comp = pd.read_csv('https://raw.githubusercontent.com/ramirezlab/PILE/main/2.%20De%20datos%20a%20gráficas%3A%20Propiedades%20drug-likeness%20y%20similitud%20química%20con%20python/data/compounds_P49841_lipinski.csv')
# Mostrar las primeras 10 filas del DataFrame para verificar la correcta importación de los datos
lipinski_comp.head(10)

Debido a que las propiedades químicas de la regla de cinco están en diferentes órdenes de magnitud, necesitamos transformarlas para poder visualizarlas en el diagrama de radar. En este caso, la mejor forma es transformar los datos de tal manera que los límites de validación sean todos 5:

- MW original: 500 g/mol - MW modificado: 5 - regla: NW/100 (Masa molecular (g/mol)/100)
- HBA original: 10 - HBA modificado: 5 - regla: HBA/2 (# Aceptores de enlaces de hidrogeno/2)
- HBD original: 5 - no cambia (# Donores de enlaces de hidrógeno)
- LogP original: 5 - no cambia (LogP)

Por tanto, vamos a transformar las columnas `MW` y `HBA`, (los nuevos se agregan en las últimas columnas):

In [ ]:
# Escalar la masa molecular dividiéndola entre 100 y almacenarla en una nueva columna 'MW*100'
lipinski_comp['MW*100'] = lipinski_comp['MW'] / 100

# Escalar el número de aceptores de enlaces de hidrógeno dividiéndolo entre 2 y almacenarlo en 'HBA*2'
lipinski_comp['HBA*2'] = lipinski_comp['HBA'] / 2

# Mostrar las primeras 10 filas del DataFrame actualizado
lipinski_comp.head(10)

Para el gráfico de radar necesitamos las desviaciones media y estándar de un conjunto de datos, por lo que crearemos una función que nos permita calcular estas dos estadísticas para los valores escalados.

In [ ]:
# Calcular estadísticas (media y desviación estándar) de las propiedades de la regla de cinco escaladas
metrics_Ro5_stats_scaled = lipinski_comp[['MW*100', 'HBA*2', 'HBD', 'LogP']].agg(["mean", "std"])

# Mostrar el DataFrame con las métricas calculadas
metrics_Ro5_stats_scaled


Ahora vamos a crear la función que realiza el gráfico. El conjunto de datos debe proporcionarse como entrada.
La función escala los datos y encuentra la media y la desviación estándar para el diagrama de radar.

In [ ]:
def plot_radar(dataframe):
    from math import pi
    import numpy as np

    # ------- PARTE 0: Conjunto de datos escalados / Métricas -------
    df = dataframe.copy()  # Crear una copia del DataFrame original para no modificarlo directamente

    # Escalar la masa molecular y los aceptores de enlaces de hidrógeno
    df['MW*100'] = df['MW'] / 100
    df['HBA*2'] = df['HBA'] / 2

    # Calcular la media y la desviación estándar de las propiedades
    metrics_Ro5_stats_scaled = df[['MW*100', 'HBA*2', 'HBD', 'LogP']].agg(["mean", "std"])
    stats_mean = metrics_Ro5_stats_scaled.loc['mean']  # Extraer la media
    stats_std = metrics_Ro5_stats_scaled.loc['std']  # Extraer la desviación estándar

    # ------- PARTE 1: Crear el fondo del gráfico de radar -------
    
    # Número de variables a representar
    N = 4

    # Calcular los ángulos de cada eje en el gráfico de radar
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Cerrar el polígono

    # Inicializar la figura y los ejes polares
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, polar=True)

    # Si se desea rotar el gráfico para que el primer eje esté en la parte superior:
    # ax.set_theta_offset(pi/2)
    # ax.set_theta_direction(-1)

    # Definir las categorías del gráfico
    categories = ['MW (g/mol)*100', '# HBA*2', '# HBD', 'LogP']
    plt.xticks(angles[:-1], categories, size=14)  # Etiquetas de los ejes

    # Dibujar etiquetas del eje Y
    ax.set_rlabel_position(0)
    plt.yticks([1, 3, 5, 7], ["1", "3", "5", "7"], color="grey", size=12)
    plt.ylim(0, 7)  # Límite superior del eje Y

    # ------- PARTE 2: Agregar datos al gráfico -------

    # Datos de la media
    data = stats_mean.values
    data = np.append(data, data[0])  # Cerrar polígono
    ax.plot(angles, data, linewidth=3, linestyle='solid', color='purple', label="mean")

    # Datos de la media + desviación estándar
    data_std_up = stats_mean.values + stats_std.values
    data_std_up = np.append(data_std_up, data_std_up[0])  # Cerrar polígono
    ax.plot(angles, data_std_up, linewidth=2, linestyle='dashed', color='limegreen', label="mean + std")

    # Datos de la media - desviación estándar
    data_std_down = stats_mean.values - stats_std.values
    data_std_down = np.append(data_std_down, data_std_down[0])  # Cerrar polígono
    ax.plot(angles, data_std_down, linewidth=2, linestyle='dashed', color='limegreen', label="mean - std")

    # Agregar texto con el número total de compuestos en el conjunto de datos
    ax.text(-np.pi/3, 8, f'# Total data: {len(dataframe)}', size=14)

    # Área correspondiente a la regla de cinco de Lipinski (valores de referencia)
    ro5_properties = [5, 5, 5, 5, 5]  # Representa los límites de MW/100, HBA/2, HBD y LogP
    ax.fill(angles, ro5_properties, 'thistle', alpha=0.6, label="rule of five area")

    # Agregar leyenda
    plt.legend(loc='upper right')

    # Mostrar el gráfico
    plt.show()

In [ ]:
# Trazamos el radarplot para el conjunto de datos de compuestos (TODOS).
plot_radar(df_molecule)

### Radar plot - Rof confort: SÍ
Ahora vamos a repetir el proceso, pero solo con las moléculas que pasaron la prueba de la regla de cinco.
Primero debemos filtrar el conjunto `rule_of_five_conform: yes`

In [ ]:
# Filtrar el DataFrame para obtener solo los compuestos que cumplen con la regla de cinco de Lipinski
df_molecule_Ro5_yes = df_molecule[df_molecule['rule_of_five_conform'] == 'yes']

# Restablecer el índice del DataFrame, eliminando el índice anterior
df_molecule_Ro5_yes.reset_index(inplace=True, drop=True)

# Mostrar el conjunto de datos de compuestos que cumplen con la regla de cinco
df_molecule_Ro5_yes


Trazamos el radarplot para el conjunto de datos filtrado

In [ ]:
plot_radar(df_molecule_Ro5_yes)

## Radar plot - Rof confort: NO
Ahora vamos a repetir el proceso, pero solo con las moléculas que pasaron la prueba de la regla de cinco.
Primero debemos filtrar el conjunto `rule_of_five_conform: no`

In [ ]:
# Filtrar el DataFrame para obtener solo los compuestos que NO cumplen con la regla de cinco de Lipinski
df_molecule_Ro5_no = df_molecule[df_molecule['rule_of_five_conform'] == 'no']

# Restablecer el índice del DataFrame, eliminando el índice anterior
df_molecule_Ro5_no.reset_index(inplace=True, drop=True)

# Mostrar el conjunto de datos de compuestos que NO cumplen con la regla de cinco
df_molecule_Ro5_no


Trazamos el diagrama de radar para el conjunto de datos de compuestos que violan el Ro5

In [ ]:
plot_radar(df_molecule_Ro5_no)

##  Estimación cuantitativa de la similitud con un fármaco (QED)

El QED es un descriptor utilizado para medir la similitud farmacológica de un compuesto. Aunque la regla de cinco (Ro5) se emplea comúnmente con este propósito, no todos los fármacos cumplen estrictamente con sus criterios. Aproximadamente el 16% de los medicamentos orales incumplen al menos uno de los parámetros de Ro5, y un 6% no satisface dos o más. Para cuantificar la calidad de los compuestos, se aplica el concepto de deseabilidad, lo que permite establecer una métrica cuantitativa denominada QED (Estimación Cuantitativa de la Similitud con los Fármacos). Los valores de QED oscilan entre cero, indicando la presencia de todas las propiedades desfavorables, y uno, representando un perfil con todas las propiedades favorables.

In [3]:
# Importar el conjunto de datos desde la URL y cargarlo en un DataFrame de pandas
lipinski_comp = pd.read_csv('https://raw.githubusercontent.com/ramirezlab/PILE/main/2.%20De%20datos%20a%20gráficas%3A%20Propiedades%20drug-likeness%20y%20similitud%20química%20con%20python/data/compounds_P49841_lipinski.csv')

# Mostrar las primeras 10 filas del DataFrame para verificar la correcta importación de los datos
lipinski_comp.head(10)

UnicodeEncodeError: 'ascii' codec can't encode character '\xe1' in position 50: ordinal not in range(128)

## Actividad práctica

Teniendo en cuenta lo revisado en esta segunda parte, realice un codigo en python con el cual pueda:

1. Hacer un gráfico de barras para los últimos 5 compuestos que no cumplen con la regla de Lipinski, en el cual se observe cada compuesto frente a las 4 reglas de Lipinski con sus respectivos límites.

Al finalizar deberá preparar un documento en formato "ipynb" (Jupyter notebook)" en el cual adjunte:
1. El código propuesto para la selección de los compuestos a gráficar con su recpectiva salida. 
2. El código propuesto para realizar el gráfico solicitado con su respectiva salida.

## Conclusión

En esta práctica, hemos aprendido sobre la regla de Lipinski como una medida para estimar la biodisponibilidad oral de un compuesto y hemos aplicado la regla en un conjunto de datos para poder filtrarlos y descartar aquellos compuestos que cumplen con dos o más de los criterios. Además, aprendemos a realizar gráficos sencillos como los de barras que nos permiten visualizar el conjunto de datos en total o cada compuesto del conjunto de datos. También, aprendimos a realizar, gráficos de dispersión que nos permiten observar el conjunto de datos frente a los cuatro criterios de la regla de Lipinski. Finalmente, construimos un gráfico más complejo como el gráfico de radar el cual nos permite comparar múltiples variables (reglas de Lipinski) en un solo gráfico.

# Referencias
1. Grogan, S., & Preuss, C. V. (2022). Pharmacokinetics. En StatPearls. StatPearls Publishing. http://www.ncbi.nlm.nih.gov/books/NBK557744/
2. Doogue, M. P., & Polasek, T. M. (2013). The ABCD of clinical pharmacokinetics. Therapeutic Advances in Drug Safety, 4(1), 5-7. https://doi.org/10.1177/2042098612469335
3. Turner, J. V., & Agatonovic-Kustrin, S. (2007). In silico prediction of oral bioavailability. En Comprehensive Medicinal Chemistry II (pp. 699-724). Elsevier. https://doi.org/10.1016/B0-08-045044-X/00147-4
